In [2]:
import torch
import torch.nn as nn

In [18]:
class FCN32s(nn.Module):
  def __init__(self,num_classes=21):
    super(FCN32s,self).__init__()

    def CBR(in_channels,out_channels,kernel_size=3,stride=1,padding=1):
      return nn.Sequential(nn.Conv2d(in_channels=in_channels,
                                     out_channels=out_channels,
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=padding),
                           nn.ReLU(inplace=True))
    # conv1
    self.conv1_1 = CBR(3,64,3,1,1)
    self.conv1_2 = CBR(64,64,3,1,1)
    self.pool1 = nn.MaxPool2d(2,stride=2,ceil_mode=True)# ceil_mode = True -> if 7x7 input , 4x4 output / ceil_mode = False -> if 7x7 input , 3x3 output

    # conv2
    self.conv2_1 = CBR(64,128,3,1,1)
    self.conv2_2 = CBR(128,128,3,1,1)
    self.pool2 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

    # conv3
    self.conv3_1 = CBR(128,256,3,1,1)
    self.conv3_2 = CBR(256,256,3,1,1)
    self.conv3_3 = CBR(256,256,3,1,1)
    self.pool3 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

    # conv4
    self.conv4_1 = CBR(256,512,3,1,1)
    self.conv4_2 = CBR(512,512,3,1,1)
    self.conv4_3 = CBR(512,512,3,1,1)
    self.pool4 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

    # conv5
    self.conv5_1 = CBR(512,512,3,1,1)
    self.conv5_2 = CBR(512,512,3,1,1)
    self.conv5_3 = CBR(512,512,3,1,1)
    self.pool5 = nn.MaxPool2d(2,stride=2,ceil_mode=True)

    # fc6
    self.fc6 = CBR(512,4096,1,1,0)
    self.drop6 = nn.Dropout2d()

    # fc7
    self.fc7 = CBR(4096,4096,1,1,0)
    self.drop7 = nn.Dropout2d()

    # Score
    self.score_fr = nn.Conv2d(4096,num_classes,kernel_size=1,stride=1,padding=0)

    # UPScore using deconv
    self.upscore32 = nn.ConvTranspose2d(num_classes,num_classes,kernel_size=64,stride=32,padding=16)

  def forward(self, x): # if x : 224x224
      h = self.conv1_1(x)
      h = self.conv1_2(h)
      h = self.pool1(h) # x: 112x112

      h = self.conv2_1(h)
      h = self.conv2_2(h)
      h = self.pool2(h) # x: 56x56

      h = self.conv3_1(h)
      h = self.conv3_2(h)
      h = self.conv3_3(h)        
      h = self.pool3(h) # x: 28x28

      h = self.conv4_1(h)
      h = self.conv4_2(h)
      h = self.conv4_3(h)        
      h = self.pool4(h) # x: 14x14

      h = self.conv5_1(h)
      h = self.conv5_2(h)
      h = self.conv5_3(h)        
      h = self.pool5(h) # x: 7x7
      
      h = self.fc6(h)
      h = self.drop6(h) # x: 7x7

      h = self.fc7(h)
      h = self.drop7(h) # x: 7x7
      
      h = self.score_fr(h)
      upscore32 = self.upscore32(h)  # 224x224
      
      return upscore32

In [19]:
# 구현된 model에 임의의 input을 넣어 output이 잘 나오는지 test

model = FCN32s(num_classes=12)
x = torch.randn((10, 3, 512, 512))
print("input shape : ", x.shape)
out = model(x)
print("output shape : ", out.size())

input shape :  torch.Size([10, 3, 512, 512])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


output shape :  torch.Size([10, 12, 512, 512])
